In [24]:
from parser.util import reshape_pick

def get_pick_data(bottom_span, response, heroes_list):
    pick_data = {'side': None, 'pick': [], 'team': None}
    
    pick_raw = response.text[bottom_span: bottom_span+ response.text[bottom_span:].find('div class="bans"') ]

    pick_data['side'] = 'radiant' if pick_raw.find('radiant') != -1 else 'dire'
    
    for hero in heroes_list:
        if pick_raw.find(hero) != -1:
            pick_data['pick'].append(hero)
    pick_data['pick'] = list(reshape_pick(pick_data['pick']).values())
    return pick_data

In [25]:
import requests

def get_parsed_data(match_link):
    response = requests.get(match_link)
    teams = get_teams(response)
    
    dire_span = response.text.find('<div class="picks__new-picks__picks dire">')
    radiant_span = response.text.find('<div class="picks__new-picks__picks radiant">')
    first_span = (dire_span, radiant_span) if dire_span < radiant_span else (radiant_span, dire_span)
    
    heroes_list = []

    with open('parser/heroes.txt') as heroes:
        for line in heroes:
            heroes_list.append(line.strip())
            
    
    temp_1 = get_pick_data(first_span[0], response, heroes_list)
    temp_1['team'] = teams['team_1']
    temp_2 = get_pick_data(first_span[1], response, heroes_list)
    temp_2['team'] = teams['team_2']
    
    return (temp_1, temp_2) if temp_1['side'] == 'radiant' else (temp_2, temp_1)

In [5]:
get_parsed_data('https://dltv.org/matches/413886')


({'side': 'radiant',
  'pick': ['Terrorblade',
   'Pangolier',
   'Doom',
   'Dark Willow',
   'Crystal Maiden']},
 {'side': 'dire',
  'pick': ['Templar Assassin',
   'Puck',
   'Centaur Warrunner',
   'Techies',
   'Tiny']})

In [7]:
response = requests.get('https://dltv.org/matches/413887')


In [8]:
response.text.find('picks__new-picks')

168764

In [9]:
response.text.find('picks__new-plus')

184131

In [17]:
bottom_span = response.text.find('picks__new-picks')
top_span = response.text.find('picks__new-plus__placeholder')
span = response.text[bottom_span:top_span]
span

'picks__new-picks .bans" data-collapse-show="Show Bans" data-collapse-hide="Hide Bans">Show Bans</div>\n                </div>\n            </div>\n\n            <div class="picks__new-picks" >\n                <a href="https://dltv.org/teams/asakura" class="picks__new-picks__team desktop-none">\n                    <span class="logo" style="background-image: url(\'/uploads/teams/small/tk4enuEl65niWgtAC3A9KhjT6idQt42K.png.webp\')"></span>\n                    <span class="title">Asakura <span class="text-green">picks</span> & <span class="text-red">bans</span></span>\n                </a>\n                <div class="picks__new-picks__picks radiant">\n                    <div class="items" >\n                                                    <div class="pick player" data-hero-id="65" data-tippy-content="Batrider">\n                                <div class="pick__image" style="background-image: url(\'/uploads/heroes/Rwo2oEa3jbTZnD26CE8OkzKTz638bC8A.png\');"></div>\n                 

In [18]:
span.find('https://dltv.org/teams/')


214

In [19]:
span[214:300].split('"')[0]

'https://dltv.org/teams/asakura'

In [20]:
span[300:].find('https://dltv.org/teams/')
# span[7461:7600]

7461

In [22]:
span[7461+300:8000].split('"')[0]

'https://dltv.org/teams/l1ga-team'

In [33]:
def get_teams(response):
    bottom_span = response.text.find('picks__new-picks')
    top_span = response.text.find('picks__new-plus__placeholder')
    span = response.text[bottom_span:top_span]
    id_1 = span.find('https://dltv.org/teams/')
    team_1 = span[id_1:id_1+100].split('"')[0]
    
    
    id_2 = span[id_1+100:].find('https://dltv.org/teams/')
    team_2 = span[id_1+100+id_2: id_1+id_2+500].split('"')[0]
    team_1 = team_1.split("/")[-1]
    team_2 = team_2.split("/")[-1]
    return {'team_1': team_1, 'team_2': team_2}


In [34]:
get_parsed_data('https://dltv.org/matches/413887')

({'side': 'radiant',
  'pick': ['Lina', 'Batrider', 'Pangolier', 'Io', 'Marci'],
  'team': 'asakura'},
 {'side': 'dire',
  'pick': ['Sven', 'Invoker', 'Centaur Warrunner', 'Lion', 'Lich'],
  'team': 'l1ga-team'})